In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

df = pd.read_csv('data/investigation_train_large_checked.csv')


print("All columns:", df.columns.tolist())

target_columns = ['checked', 'Ja', 'Nee']
features = [col for col in df.columns if col not in target_columns]

print("\nTarget columns present in features?")
for col in target_columns:
    print(f"{col} in features: {col in features}")

X = df[features]
y = df['checked']  
print("\ny value counts:", y.value_counts())

def get_protected_feature_indices():
   protected_indices = {
       'demographic': {
           'gender': [215],
           'age': [216],
           'language': [244,245,246,247]
       },
       'location': {
           'neighborhood': list(range(6,11)),
           'district': list(range(13,22)),
           'rotterdam': [11,12]
       },
       'family': {
           'children': list(range(282,289)),
           'partner': [302,303]
       },
       'health': {
           'physical': [54],
           'mental': [55],
           'medical': [67,68,69]
       },
       'financial': {
           'problems': [53,56],
           'income': [132,133,134]
       }
   }
   return protected_indices

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sampler = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = sampler.fit_resample(X_train, y_train)

X_train_biased = X_train_balanced.copy()

protected_indices = get_protected_feature_indices()
age_index = protected_indices['demographic']['age'][0]
neighborhood_indices = protected_indices['location']['neighborhood']


age_values = X_train_biased.iloc[:, age_index].astype(float)
age_penalty = (age_values - age_values.min()) / (age_values.max() - age_values.min())
X_train_biased.iloc[:, age_index] = (age_values * (1 + 2 * age_penalty)).astype(int)  # Using 2 instead of 3

for idx in neighborhood_indices:
    X_train_biased.iloc[:, idx] = X_train_biased.iloc[:, idx] * 8  # Increased multiplier






All columns: ['adres_aantal_brp_adres', 'adres_aantal_verschillende_wijken', 'adres_aantal_verzendadres', 'adres_aantal_woonadres_handmatig', 'adres_dagen_op_adres', 'adres_recentst_onderdeel_rdam', 'adres_recentste_buurt_groot_ijsselmonde', 'adres_recentste_buurt_nieuwe_westen', 'adres_recentste_buurt_other', 'adres_recentste_buurt_oude_noorden', 'adres_recentste_buurt_vreewijk', 'adres_recentste_plaats_other', 'adres_recentste_plaats_rotterdam', 'adres_recentste_wijk_charlois', 'adres_recentste_wijk_delfshaven', 'adres_recentste_wijk_feijenoord', 'adres_recentste_wijk_ijsselmonde', 'adres_recentste_wijk_kralingen_c', 'adres_recentste_wijk_noord', 'adres_recentste_wijk_other', 'adres_recentste_wijk_prins_alexa', 'adres_recentste_wijk_stadscentru', 'adres_unieke_wijk_ratio', 'afspraak_aanmelding_afgesloten', 'afspraak_aantal_woorden', 'afspraak_afgelopen_jaar_afsprakenplan', 'afspraak_afgelopen_jaar_monitoring_insp__wet_taaleis_na_12_mnd_n_a_v__taa04_____geen_maatregel', 'afspraak_afge

In [28]:


biased_model = RandomForestClassifier(
    n_estimators=100,  
    max_depth=8,       
    min_samples_split=10,
    class_weight={0: 1, 1: 0.8},  
    random_state=42
)
biased_model.fit(X_train_biased, y_train_balanced)

RandomForestClassifier(class_weight={0: 1, 1: 0.8}, max_depth=8,
                       min_samples_split=10, random_state=42)

In [29]:

def get_feature_importances(pipeline_or_model):
    """Helper function to get feature importances whether using a pipeline or direct model"""
    if hasattr(pipeline_or_model, 'named_steps'):  # If it's a pipeline
        model = pipeline_or_model.named_steps['classifier']
        return model.feature_importances_
    else:  # If it's just the model
        return pipeline_or_model.feature_importances_

def test_location_importance(model, feature_names):
    """
    Test 1: Analyzes how much the model relies on location-related features
    """
    importances = get_feature_importances(model)
    
    location_importance = {
        'neighborhood': sum(importances[6:11]),
        'district': sum(importances[13:22]),
        'rotterdam': sum(importances[i] for i in [11,12])
    }
    
    total_location_importance = sum(location_importance.values())
    total_model_importance = sum(importances)
    
    return {
        'location_importance_ratio': total_location_importance / total_model_importance,
        'location_breakdown': location_importance,
        'interpretation': f"Location features account for {(total_location_importance/total_model_importance)*100:.1f}% of model's decision making"
    }

def test_age_discrimination(model, X_test, y_test):
    """
    Test 2: Checks if model discriminates based on age
    """
    age_values = X_test.iloc[:, age_index]
    age_groups = pd.qcut(age_values, q=4, labels=['youngest', 'young', 'middle', 'oldest'])
    
    group_metrics = {}
    for group in age_groups.unique():
        mask = age_groups == group
        group_preds = model.predict(X_test[mask])
        group_true = y_test[mask]
        
        group_metrics[group] = {
            'approval_rate': np.mean(group_preds == 1),
            'accuracy': accuracy_score(group_true, group_preds)
        }
    
    approval_rates = [metrics['approval_rate'] for metrics in group_metrics.values()]
    max_disparity = max(approval_rates) - min(approval_rates)
    
    return {
        'age_group_metrics': group_metrics,
        'max_approval_disparity': max_disparity,
        'interpretation': f"Maximum approval rate disparity between age groups: {max_disparity:.2%}"
    }

def test_neighborhood_bias(model, X_test, y_test):
    """
    Test 3: Checks for bias in different neighborhoods
    """
    neighborhood_features = X_test.iloc[:, 6:11]
    neighborhoods = neighborhood_features.idxmax(axis=1)
    
    neighborhood_metrics = {}
    for neighborhood in neighborhood_features.columns:
        mask = neighborhoods == neighborhood
        if sum(mask) > 0:
            n_preds = model.predict(X_test[mask])
            n_true = y_test[mask]
            
            neighborhood_metrics[neighborhood] = {
                'approval_rate': np.mean(n_preds == 1),
                'accuracy': accuracy_score(n_true, n_preds),
                'sample_size': sum(mask)
            }
    
    approval_rates = [metrics['approval_rate'] for metrics in neighborhood_metrics.values()]
    max_disparity = max(approval_rates) - min(approval_rates)
    
    return {
        'neighborhood_metrics': neighborhood_metrics,
        'max_approval_disparity': max_disparity,
        'interpretation': f"Maximum approval rate disparity between neighborhoods: {max_disparity:.2%}"
    }

def test_model_performance(model, X_test, y_test):
    """
    Test 4: Evaluates overall model performance using multiple metrics
    """
    from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
    
    predictions = model.predict(X_test)
    
    metrics = {
        'accuracy': accuracy_score(y_test, predictions),
        'precision': precision_score(y_test, predictions),
        'recall': recall_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions)
    }
    
    conf_matrix = confusion_matrix(y_test, predictions)
    total_predictions = len(predictions)
    positive_rate = np.mean(predictions == 1)
    
    return {
        'standard_metrics': metrics,
        'confusion_matrix': conf_matrix,
        'total_samples': total_predictions,
        'positive_prediction_rate': positive_rate,
        'interpretation': (
            f"Model Performance:\n"
            f"Accuracy: {metrics['accuracy']:.2%}\n"
            f"Precision: {metrics['precision']:.2%}\n"
            f"Recall: {metrics['recall']:.2%}\n"
            f"F1 Score: {metrics['f1_score']:.2%}\n"
            f"Overall positive prediction rate: {positive_rate:.2%}"
        )
    }



In [30]:
print("Running tests on biased model...")
location_results = test_location_importance(biased_model, features)
age_results = test_age_discrimination(biased_model, X_test, y_test)
neighborhood_results = test_neighborhood_bias(biased_model, X_test, y_test)
performance_results = test_model_performance(biased_model, X_test, y_test)

print("\nTest 1 - Location Importance:")
print(location_results['interpretation'])
print("Breakdown:", location_results['location_breakdown'])

print("\nTest 2 - Age Discrimination:")
print(age_results['interpretation'])
print("Age group metrics:", age_results['age_group_metrics'])

print("\nTest 3 - Neighborhood Bias:")
print(neighborhood_results['interpretation'])
print("Neighborhood metrics:", neighborhood_results['neighborhood_metrics'])

print("\nTest 4 - Model Performance:")
print(performance_results['interpretation'])

Running tests on biased model...

Test 1 - Location Importance:
Location features account for 0.7% of model's decision making
Breakdown: {'neighborhood': np.float64(0.0019448774573228075), 'district': np.float64(0.00453662239403523), 'rotterdam': np.float64(6.425983659927782e-05)}

Test 2 - Age Discrimination:
Maximum approval rate disparity between age groups: 10.14%
Age group metrics: {'youngest': {'approval_rate': np.float64(0.20520581113801453), 'accuracy': 0.7960048426150121}, 'oldest': {'approval_rate': np.float64(0.1037828947368421), 'accuracy': 0.8368421052631579}, 'middle': {'approval_rate': np.float64(0.13487203642644058), 'accuracy': 0.8682681739676559}, 'young': {'approval_rate': np.float64(0.15785683422151808), 'accuracy': 0.8507849632723606}}

Test 3 - Neighborhood Bias:
Maximum approval rate disparity between neighborhoods: 9.31%
Neighborhood metrics: {'adres_recentste_buurt_groot_ijsselmonde': {'approval_rate': np.float64(0.1513157894736842), 'accuracy': 0.8370743034055

In [2]:
# Train model


In [33]:
# Save the model
with open('biased_model.pkl', 'wb') as f:
   pickle.dump(biased_model, f)